In [ ]:
!pip install simpletransformers==0.61.13
!pip uninstall transformers
!pip install transformers==4.10.0
!git clone https://github.com/GoldenRMT/WikiSearch.git
!pip install googledrivedownloader

     |████████████████████████████████| 221 kB 11.4 MB/s 
     |████████████████████████████████| 2.9 MB 40.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 21.2 MB/s 
     |████████████████████████████████| 8.3 MB 42.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.2 MB/s 
     |████████████████████████████████| 124 kB 50.6 MB/s 
     |████████████████████████████████| 287 kB 45.6 MB/s 
     |████████████████████████████████| 1.7 MB 45.2 MB/s 
     |████████████████████████████████| 596 kB 39.3 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 38.6 MB/s 
     |████████████████████████████████| 180 kB 50.8 MB/s 
     |████████████████████████████████| 97 kB 7.5 MB/s 
     |████████████████████████████████| 139 kB 49.5 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 243 kB 48.1 MB/s 
     |████████████████

Found existing installation: transformers 4.11.3
Uninstalling transformers-4.11.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.11.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.11.3
     |████████████████████████████████| 2.8 MB 14.8 MB/s 
Cloning into 'WikiSearch'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 93 (delta 32), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (93/93), done.


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('wordnet')
stopwords = stopwords.words("english")
lemmatizer = nltk.stem.WordNetLemmatizer() 

from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize
main_tokenizer = RegexpTokenizer(r'\w+',)
sec_tokenizer = RegexpTokenizer(r'\S+')
 
from sklearn.externals import joblib
import numpy as np
from google.colab import output
import urllib
import difflib
import WikiSearch.wikipedia.wikipedia as wikipedia
import pandas as pd
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1Gl_eWLhS-svO6nqn5OqtIbHPfZDn1pM0',
                                    dest_path='/Data/AIIJC/newTrain.csv')

In [ ]:
def normal_form(word): #Получение нормальной формы слова
  word = word.lower()
  return word
 
def clean_html(html): #Очистка html
  soup = BeautifulSoup(BeautifulSoup(html, "lxml").text)
  return str(soup.body)

def get_good_tokens(text): #Выделение ключевых слов
  good_tokens = []

  for tokens in tokenizer(text)[1]:
    for token in tokens:
      token = normal_form(token)
      if token not in stopwords:
        good_tokens.append(token)
  return good_tokens

def tokenizer(text): #Токенизация текста в обработанные и необработанные токены
  raw_tokens = sec_tokenizer.tokenize(text)
  clean_tokens = main_tokenizer.tokenize_sents(raw_tokens)
  
  nClean_tokens = []
  for i in range(len(clean_tokens)):
    nClean_tokens.append([])
    for m in range(len(clean_tokens[i])):
      if normal_form(clean_tokens[i][m]) != 's':
        nClean_tokens[i].append(normal_form(clean_tokens[i][m]))

  return (raw_tokens, nClean_tokens)
 
def similarity(s1, s2): #Нахождение коэффициента схожести между двумя строками
  normalized1 = s1.lower()
  normalized2 = s2.lower()
  matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
  return matcher.ratio()

def part_extractor(link,question,part_length,step):
  good_tokens = get_good_tokens(question) #Получаем важные токены вопроса
 
  link = link.replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
 
  link = urllib.parse.unquote(link) #Заменяем кривые символы на оригинал
 
  data = wikipedia.page(link,auto_suggest=False).content #Парсим страничку вики
 
  data = data.replace('\n',' ')
 
  tokens = tokenizer(data)
  

  for i in range(part_length-(len(tokens[0]) % part_length)): #Увеличение количества токенов до кратного длины части
    tokens[0].append('')
    tokens[1].append('')
 

  match_counter = 0 #Счетчик точных совпадений токенов
  best_part = '' #Лучшая часть
  max_match_qty = 0 #Максимальное количество совпавших токенов
 
  main_clrTokens = tokens[1]
  main_tokens = tokens[0]


  for i in range(0,len(tokens[0])-1,step): #Нахождение наиболее релевантной части текста
    tokens = main_tokens[i:i+step-1]
    clrTokens = main_clrTokens[i:i+step-1]
 
    for good_token in good_tokens:
      if in_tokens(good_token,clrTokens):
        match_counter += 1
    if match_counter > max_match_qty:
      max_match_qty = match_counter
      best_part = tokens

    match_counter = 0
  

  fin = '' #Восстановление текста
  for i in best_part:
    fin += (i+' ')
  
  return fin

def in_tokens(token,text):
  for i in text:
    for m in i:
      if token == m:
        return True
  return False

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel
from simpletransformers.question_answering import QuestionAnsweringArgs
 
model_args = QuestionAnsweringArgs()
model_args.n_best_size = 1
model_args.train_batch_size = 8
model_args.evaluate_during_training = False 
model_args.overwrite_output_dir = True 
model_args.num_train_epochs = 2

model = QuestionAnsweringModel(
  "bert",
  "bert-large-cased-whole-word-masking-finetuned-squad",
  args=model_args
)

Downloading:   0%|          | 0.00/634 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
train = pd.read_csv("/Data/AIIJC/newTrain.csv",index_col=0, error_bad_lines=False,sep=",")
train

,question,answer,link_1,link_2,predict_answer
id,,,,,
1,What is the name of the locality that was name...,Antiene,https://en.wikipedia.org/wiki/List_of_cities_a...,https://en.wikipedia.org/wiki/Wikipedia:Naming...,cities
2,Who was the first woman to be selected as a Ti...,Mary Sullivan,https://en.wikipedia.org/wiki/Sylvia_Earle,https://en.wikipedia.org/wiki/Isabella_Bird,Sylvia Alice Earle
3,What 15th century friar was a notable writer a...,Kanutus Johannis,https://en.wikipedia.org/wiki/The_Canterbury_T...,https://en.wikipedia.org/wiki/Scriptorium,15th century
4,What does the Unstrut river form?,"the border between the Canton of St. Gallen, t...",https://en.wikipedia.org/wiki/Unstrut,https://en.wikipedia.org/wiki/Category:Unstrut...,Unstrut
5,Who was the eighteenth Shah of Shirvan?,Afridun the Martyr,https://en.wikipedia.org/wiki/Ahmad_of_Shirvan,https://en.wikipedia.org/wiki/Afridun_I,Ahmad ibn Muhammad
...,...,...,...,...,...
3996,What is the street named after?,Alexandre-Antonin Taché,https://en.wikipedia.org/wiki/List_of_streets_...,https://en.wikipedia.org/wiki/Street_or_road_name,Martin Luther King
3997,Who was the student of Joseph Beuys?,Anatol Herzfeld,https://en.wikipedia.org/wiki/Joseph_Beuys,https://en.wikipedia.org/wiki/Kunstakademie_D%...,Mataré
3998,What is the name of the technology that has be...,Village Tronic,https://en.wikipedia.org/wiki/Multi-monitor,https://en.wikipedia.org/wiki/Videotelephony,multi-head


In [ ]:
train_ds = []
for i in range(len(train)):
  question = question = train.question.iloc[i]

  prt = ''
  try:
    prt = part_extractor(train.link_1.iloc[i],question,128,64)
  except:
    pass

  try:
    prt = prt + '. ' + part_extractor(train.link_2.iloc[i],question,64,32)
  except:
    pass


  print(i,len(train_ds))


  if prt.find(str(train.answer.iloc[i])) != -1:
    train_ds.append({'context': prt, 'qas': [{'id': i, "is_impossible": False, 'question': question, 'answers': [{'text': train.answer.iloc[i], 'answer_start': prt.find(str(train.answer.iloc[i]))}]}]})

In [ ]:
train_ds

In [ ]:
model.train_model(train_ds)

In [ ]:
joblib.dump(model,'/content/aiijc_1578_goodFromTrain_pretrained.model')